In [53]:
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD

data: np.ndarray = np.genfromtxt('apartmentComplexData.txt', delimiter=',')
variables = data[:, 0:8]
values = data[:, 8]

model = Sequential()
model.add(Dense(8, input_shape=(8,)))
model.add(Activation('linear'))

print(model.summary)


sgd = SGD(0.01)

model.compile(loss='mse', optimizer=sgd, metrics=['mse'])

H = model.fit(variables, values, epochs=10)
W = model.get_weights()


<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x7f5ab0800eb8>>
Epoch 1/10
645/645 [==============================] - 1s 1ms/step - loss: nan - mse: nan
Epoch 2/10
645/645 [==============================] - 1s 1ms/step - loss: nan - mse: nan
Epoch 3/10
645/645 [==============================] - 1s 1ms/step - loss: nan - mse: nan
Epoch 4/10
645/645 [==============================] - 1s 1ms/step - loss: nan - mse: nan
Epoch 5/10
645/645 [==============================] - 1s 1ms/step - loss: nan - mse: nan
Epoch 6/10
645/645 [==============================] - 1s 1ms/step - loss: nan - mse: nan
Epoch 7/10
645/645 [==============================] - 1s 1ms/step - loss: nan - mse: nan
Epoch 8/10
645/645 [==============================] - 1s 1ms/step - loss: nan - mse: nan
Epoch 9/10
600/645 [==========================>...] - ETA: 0s - loss: nan - mse: nan

KeyboardInterrupt: ignored

In [180]:
import keras 
import numpy as np
from keras.datasets import boston_housing 
from keras.models import Sequential 
from keras.layers import Dense 
from keras.optimizers import RMSprop 
from keras.callbacks import EarlyStopping 
from sklearn import preprocessing 
from sklearn.preprocessing import scale
import pandas as pd

pd_data = pd.read_csv(
    'apartmentComplexData.txt', 
    delimiter=',',
    names=['1', '2', '3', '4', '5', '6', '7', '8', '9']    
)
pd_data = pd_data.drop(columns=['1', '2', '8'])
data = pd_data.to_numpy()

train_data = data[: int(len(data) * 0.75)]
train_variables = train_data[:, :5]
train_values = train_data[:, 5]

test_data = data[int(len(data) * 0.75) :]
test_variables = test_data[:, :5]
test_values = test_data[:, 5]

x_train_scaled = preprocessing.scale(train_variables) 
scaler = preprocessing.StandardScaler().fit(train_variables) 
x_test_scaled = scaler.transform(test_variables)

model = Sequential() 
model.add(Dense(
      5,
      kernel_initializer = 'normal', 
      activation = 'relu',
      input_shape = (5,)
))
model.add(Dense(
      5,
      kernel_initializer = 'normal', 
      activation = 'relu',
      input_shape = (5,)
))
model.add(Dense(
      5,
      kernel_initializer = 'normal', 
      activation = 'relu',
      input_shape = (5,)
))
model.add(Dense(
      5,
      kernel_initializer = 'normal', 
      activation = 'relu',
      input_shape = (5,)
))
model.add(Dense(
      5,
      kernel_initializer = 'normal', 
      activation = 'relu',
      input_shape = (5,)
))
model.add(Dense(5, activation = 'relu'))
model.add(Dense(1))

sgd = keras.optimizers.SGD(learning_rate = 0.001)
model.compile(loss='mean_squared_error', optimizer=sgd, metrics=['mean_absolute_error'])

# model.compile(
#    loss = 'mse', 
#    optimizer = RMSprop(), 
#    metrics = ['mean_absolute_error']
# )


history = model.fit(
   x_train_scaled, train_values,    
   batch_size = 64, 
   epochs = 100,
   verbose = 1,
   validation_split = 0.25,
   callbacks = [EarlyStopping(monitor = 'val_loss', patience = 20)]
)


score = model.evaluate(x_train_scaled, train_values, verbose = 1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# prediction = model.predict(variables[0]) 
# print(prediction.flatten()) 
# print(y_test)



Epoch 1/100
182/182 [==============================] - 1s 3ms/step - loss: 51545634334.7760 - mean_absolute_error: 194469.7783 - val_loss: 16595280896.0000 - val_mean_absolute_error: 97004.0781
Epoch 2/100
182/182 [==============================] - 0s 2ms/step - loss: 31413770217.6175 - mean_absolute_error: 136774.1659 - val_loss: 9948985344.0000 - val_mean_absolute_error: 65304.4570
Epoch 3/100
182/182 [==============================] - 0s 2ms/step - loss: 21716285730.9727 - mean_absolute_error: 106988.8898 - val_loss: 7703605248.0000 - val_mean_absolute_error: 58512.2695
Epoch 4/100
182/182 [==============================] - 0s 2ms/step - loss: 17368502602.1421 - mean_absolute_error: 94305.5187 - val_loss: 7293515264.0000 - val_mean_absolute_error: 61727.6562
Epoch 5/100
182/182 [==============================] - 0s 2ms/step - loss: 15615372998.6448 - mean_absolute_error: 90334.8586 - val_loss: 7559501824.0000 - val_mean_absolute_error: 67122.5312
Epoch 6/100
182/182 [===============